# Missing users 9h-11h

In [3]:
import pandas as pd
import numpy as np
import os
import itertools
import random
import utm

In [3]:
input_path1 = os.getcwd() + "/OUTPUTS_all_in_city_9h_11h/path.csv"
input_path2 = os.getcwd() + "/INPUTS/all_in_city_9h_11h.csv"
Output = pd.read_csv(input_path1, delimiter = ';')
Input = pd.read_csv(input_path2, delimiter = ';')
listID_in = Input['ID'].tolist()
listID_out = Output['ID'].tolist()

In [4]:
def diff(l1, l2):   # return a list of lost users
    difference = list(set(l1).symmetric_difference(set(l2)))
    return difference

## Missing part of user

In [7]:
def missingData(inputDataframe, listID_in, listID_out):
    missing = diff(listID_in, listID_out)
    indexUser = [Input.index[Input['ID'] == i].tolist() for i in missing]  # index of missing user
    indexUser = list(itertools.chain.from_iterable(indexUser))
    return Input.iloc[indexUser]

In [8]:
df_missing = missingData(Input, listID_in, listID_out)
df_missing

,ID,DEPARTURE,ORIGIN,DESTINATION
22148,1310743,10:19:06,629516.0030987657 5803041.088138838,622134.832309049 5795422.854505211
24316,1048616,10:26:33,626750.1937311369 5803857.030713063,630576.3552932777 5798625.9269568315
9077,1048691,09:21:56,625994.6111860399 5803104.322472489,622153.8568623343 5798274.575956808
734,1638527,09:01:46,623222.3060497161 5809930.597505666,628516.0324350919 5803489.2816736465
15379,983202,09:45:58,625211.8264193003 5810221.765439114,629164.5617305066 5804166.525509925
...,...,...,...,...
30742,1113845,10:45:47,624879.7101665467 5810177.090008756,633954.0091431895 5795963.661617139
9479,1048309,09:22:55,626201.4239205915 5803154.740379039,629325.6710015375 5796760.34834227
26839,1113868,10:34:32,634268.1582760159 5797820.234783638,624804.5873931504 5809659.6469628615
2173,982871,09:05:07,626982.8294077505 5807346.114983573,624404.3026659425 5810383.893081881


In [10]:
def bounding(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    l = O + D  # Concatenate 2 2D lists
    coord = np.array([x.split() for x in l])
    Xmin = min(float(x[0]) for x in coord)
    Ymin = min(float(x[1]) for x in coord)
    Xmax = max(float(x[0]) for x in coord)
    Ymax = max(float(x[1]) for x in coord)
    return Xmin, Xmax, Ymin, Ymax

In [7]:
# Highway coordinates
x, y, zone, ut = utm.from_latlon(52.32518282260862, 4.834477934348526)
(x,y)

(625017.9576056905, 5798791.299367087)

In [8]:
x, y, zone, ut = utm.from_latlon(52.42619306976802, 4.980217278228295)
(x,y)

(634641.3677946266, 5810286.725571055)

In [20]:
utm.from_latlon(52.42619306976802, 4.980217278228295)

(634641.3677946266, 5810286.725571055, 31, 'U')

## All in city

In [9]:
def in_out_city(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    ID = df.ID.tolist()  ## no use
    array_O = np.array([x.split() for x in O])
    array_D = np.array([x.split() for x in D])
    coord_O = array_O.astype(float)
    coord_D = array_D.astype(float)
    # Bounding of city Amsterdam
    ##(xmin, ymin, xmax, ymax) = (621225.6754145394, 5795122.850697235, 635225.6754145394, 5810622.850697235)
    # (xmin, ymin, xmax, ymax) = (52.32518282260862, 4.834477934348526, 52.42619306976802, 4.980217278228295)
    (xmin, ymin, xmax, ymax) = (625017.9576056905, 5798791.299367087, 634641.3677946266, 5810286.725571055)
    In = lambda x, y : x <= xmax and x >= xmin and y <= ymax and y>= ymin  # if between xmin and xmax
    Out = lambda x, y : x > xmax or x < xmin or y > ymax or y < ymin   # if not
    (in_in, in_out, out_in, out_out) = (0, 0, 0, 0)
    for i in range(len(O)):
        if In(coord_O[i][0], coord_O[i][1]) & In(coord_D[i][0], coord_D[i][1]):
            in_in+=1
        elif In(coord_O[i][0], coord_O[i][1]) & Out(coord_D[i][0], coord_D[i][1]):
            in_out+=1
        elif Out(coord_O[i][0], coord_O[i][1]) & In(coord_D[i][0], coord_D[i][1]):
            out_in+=1
        elif Out(coord_O[i][0], coord_O[i][1]) & Out(coord_D[i][0], coord_D[i][1]):
            out_out+=1
    per_in_in = (in_in / len(df))*100
    per_in_out = (in_out / len(df))*100
    per_out_in = (out_in / len(df))*100
    per_out_out = (out_out / len(df))*100
    sum_out = in_out + out_in + out_out
    return per_in_in, per_in_out, per_out_in, per_out_out, sum_out

In [27]:
def get_utm(l):
    return [[utm.from_latlon(i[0], i[1])[0], utm.from_latlon(i[0], i[1])[1]] for i in l]

In [31]:
def distanceSquare(x1, y1, x2, y2):
    return (x1 - x2)**2 + (y1 - y2)**2

In [34]:
def nearest_highway(highway, x, y):
    h = get_utm(l)    # turn latlong to utm
    dist = [distanceSquare(x, y, i[0], i[1]) for i in h]
    minpos = dist.index(min(dist))  # get index of minimum distance
    return (h[minpos][0], h[minpos][1])

In [44]:
l = [[52.32791994290011, 4.913239361372001], [52.348521124104536, 4.969982909959437], [52.38911039306189, 4.967560455680146],
    [52.40771996733094, 4.938153295847306], [52.4233956111038, 4.877273846428637], 
     [52.3960595030744, 4.844561046121149], [52.3841084139694, 4.8444926286081955], [52.338567002294766, 4.841697415785518]]
test = get_utm(l)
x = 634035.7291712608 
y = 5798370.933920433
(x1, y1) = nearest_highway(l, x, y)
(x1, y1)

(634180.9360684787, 5801629.541588004)

In [39]:
## Modifie the departure or destination coordinates
def all_in_city(df):
    O = df.ORIGIN.tolist()  
    D = df.DESTINATION.tolist()
    ID = df.ID.tolist()
    Departure = df.DEPARTURE.tolist()
    Service = df.SERVICE.tolist()
    l_O = [x.split() for x in O]
    l_D = [x.split() for x in D]
    highway = [[52.32791994290011, 4.913239361372001], [52.348521124104536, 4.969982909959437], [52.38911039306189, 4.967560455680146],
    [52.40771996733094, 4.938153295847306], [52.4233956111038, 4.877273846428637], 
     [52.3960595030744, 4.844561046121149], [52.3841084139694, 4.8444926286081955], [52.338567002294766, 4.841697415785518]]
    index = []
    (xmin, ymin, xmax, ymax) = (625017.9576056905, 5798791.299367087, 634641.3677946266, 5810286.725571055)
    In = lambda x, y : x <= xmax and x >= xmin and y <= ymax and y>= ymin  # if between xmin and xmax
    Out = lambda x, y : x > xmax or x < xmin or y > ymax or y < ymin   # if not
    #in_x = lambda x: str(random.uniform(xmin, xmax)) if (x>xmax or x<xmin) else x
    #in_y = lambda y: str(random.uniform(ymin, ymax)) if (y>xmax or y<xmin) else y
    #in_xy = lambda x, y : nearest_highway(l, x, y)
    remove = lambda l : [l.pop(x) for x in sorted(index, reverse = True)]
    combine = lambda l : ['%s %s' % (l[x][0], l[x][1]) for x in range(len(l))]
    for i in range(len(l_O)):
        if not (In(float(l_O[i][0]), float(l_O[i][1])) & In(float(l_D[i][0]), float(l_D[i][1]))):
            if In(float(l_O[i][0]), float(l_O[i][1])) & Out(float(l_D[i][0]), float(l_D[i][1])):
                #l_D[i][0] = str(in_x(float(l_D[i][0])))
                #l_D[i][1] = str(in_y(float(l_D[i][1])))
                (X, Y) = nearest_highway(highway, float(l_D[i][0]),  float(l_D[i][1]))
                l_D[i][0] = str(X)
                l_D[i][1] = str(Y)
            elif Out(float(l_O[i][0]), float(l_O[i][1])) & In(float(l_D[i][0]), float(l_D[i][1])):
                #l_O[i][0] = str(in_x(float(l_O[i][0])))
                #l_O[i][1] = str(in_y(float(l_O[i][1])))
                (X, Y) = nearest_highway(highway, float(l_O[i][0]),  float(l_O[i][1]))
                l_O[i][0] = str(X)
                l_O[i][1] = str(Y)
            else:        # out_out -> remove all
                index.append(i)
    remove(ID)
    remove(l_O) 
    remove(l_D)
    remove(Departure)
    remove(Service)
    l_O = combine(l_O)
    l_D = combine(l_D)
    tmp = list(zip(ID, Departure, l_O, l_D, Service))
    df = pd.DataFrame(tmp, columns = ['ID', 'DEPARTURE', 'ORIGIN', 'DESTINATION', 'SERVICE'])
    return df

### Percentage of users found between 9h-11h

In [21]:
idOut = Output.ID.tolist()
idIn = Input.ID.to_list()
print ("percentage of users found: " + "{0:.0%}".format(len(set(idOut)) / len(set(idIn)))) 

percentage of users found: 97%


In [22]:
Output

,ID,COST,PATH,LENGTH,SERVICE
0,1019364,753.215226,ORIGIN_463 m46443108 m46443464 m46443128 m4644...,8734.373914,PersonalVehicle
1,958538,1598.196449,ORIGIN_522 m285508303 m46587350 m46587506 m465...,20466.281623,PersonalVehicle
2,1015419,433.482573,ORIGIN_493 m46519132 m4629692572 m46522643 m46...,3524.382900,PersonalVehicle
3,1033909,2.734769,ORIGIN_464 m46438252 DESTINATION_464,3.883372,PersonalVehicle
4,1122860,490.406421,ORIGIN_411 m46226031 m46234700 m46234736 m4622...,5204.571438,PersonalVehicle
...,...,...,...,...,...
34424,1115944,1630.347844,ORIGIN_465 m3190234876 m1565648188 m1118708299...,13969.986944,PersonalVehicle
34425,1070965,564.211884,ORIGIN_437 m84486707 m46331616 m1525015196 m15...,3078.613741,PersonalVehicle
34426,1094097,323.846999,ORIGIN_437 m84486707 m46331616 m1525015196 m15...,2956.862039,PersonalVehicle
34427,1084155,306.861601,ORIGIN_407 m4633328552 m46234500 m46234778 m46...,2994.476462,PersonalVehicle


## 7h-9h

In [11]:
test_7h_9h = pd.read_csv(os.getcwd() + "/INPUTS/test_7h_9h.csv", delimiter = ';')
(a,b,c,d,e) = in_out_city(test_7h_9h)
print("in_in_city :", "{0:.0f}%".format(a), "in_out_city :", "{0:.0f}%".format(b), "out_in_city :", "{0:.0f}%".format(c), "out_out_city :","{0:.0f}%".format(d), "sum_out :", e)

in_in_city : 24% in_out_city : 11% out_in_city : 15% out_out_city : 50% sum_out : 14497


In [12]:
test_7h_9h

,ID,DEPARTURE,ORIGIN,DESTINATION,SERVICE
0,1111241,07:00:00,634035.7291712608 5798370.933920433,633013.7429947932 5797843.393080657,PersonalVehicle
1,1168071,07:00:00,646689.5975135324 5796305.536300822,625022.1658398311 5803258.391749226,PersonalVehicle
2,1163767,07:00:01,647946.4852670499 5793206.220630613,650698.8689544763 5804828.471933132,PersonalVehicle
3,1040303,07:00:02,626592.2832221197 5802782.201807101,627119.3937449739 5802857.838660972,PersonalVehicle
4,1135295,07:00:03,626373.3503814898 5795318.412810527,649225.0246550543 5787834.954252227,PersonalVehicle
...,...,...,...,...,...
18990,973567,08:59:59,622484.8367840601 5815526.01778025,622336.0324728701 5815165.884126046,PersonalVehicle
18991,669847,08:59:59,671144.401814134 5756828.440589001,687346.7403420567 5742119.76719328,PersonalVehicle
18992,382377,08:59:59,669269.0467706273 5816633.331821069,667309.4649672995 5819659.861915671,PersonalVehicle
18993,1084683,08:59:59,627694.0701901402 5801284.886931946,627651.4043159435 5801055.3357389495,PersonalVehicle


In [40]:
test_all_in = all_in_city(test_7h_9h)
(a,b,c,d,e) = in_out_city(test_all_in)
print("in_in_city :", "{0:.0f}%".format(a), "in_out_city :", "{0:.0f}%".format(b), "out_in_city :", "{0:.0f}%".format(c), "out_out_city :","{0:.0f}%".format(d), "sum_out :", e)

in_in_city : 100% in_out_city : 0% out_in_city : 0% out_out_city : 0% sum_out : 0


In [41]:
test_all_in

,ID,DEPARTURE,ORIGIN,DESTINATION,SERVICE
0,1168071,07:00:00,634180.9360684787 5801629.541588004,625022.1658398311 5803258.391749226,PersonalVehicle
1,1040303,07:00:02,626592.2832221197 5802782.201807101,627119.3937449739 5802857.838660972,PersonalVehicle
2,1034483,07:00:04,627345.0816454801 5804263.484180098,634180.9360684787 5801629.541588004,PersonalVehicle
3,985601,07:00:05,634180.9360684787 5801629.541588004,628408.4006607257 5802607.606304967,PersonalVehicle
4,396330,07:00:09,634180.9360684787 5801629.541588004,629039.4545156191 5804289.138790642,PersonalVehicle
...,...,...,...,...,...
9405,1050286,08:59:56,626124.6513297992 5802432.080601969,634180.9360684787 5801629.541588004,PersonalVehicle
9406,1106874,08:59:56,632778.5268014283 5803575.269459474,634180.9360684787 5801629.541588004,PersonalVehicle
9407,1120808,08:59:57,634180.9360684787 5801629.541588004,634037.5281292694 5800649.76836723,PersonalVehicle
9408,1089329,08:59:58,627184.6443263115 5799024.982075843,626527.037702565 5799611.930337115,PersonalVehicle


In [42]:
test = test_all_in
test.to_csv('test_all_in_highway_7h_9h.csv', sep = ';', index = False)

In [29]:
def percentage_uber(p, demande):
    ID = demande['ID'].tolist()
    departure = demande['DEPARTURE'].tolist()
    origin = demande['ORIGIN'].tolist()
    destination = demande['DESTINATION'].tolist()
    service = demande['SERVICE'].tolist()
    l = list(range(len(demande)))
    indices = random.sample(l, int(p*len(demande)))
    for i in indices:
        service[i] = "UBER"
    tmp = list(zip(ID, departure, origin, destination, service))
    df = pd.DataFrame(tmp, columns = ['ID', 'DEPARTURE', 'ORIGIN', 'DESTINATION', 'SERVICE'])
    return df

In [83]:
s = [1,2,3,4,5,5]
a = [0,2,5]
m = [0,0,0]
#map(s.__setitem__, a, m)
list(map(lambda x: x.replace(s[i], m[i]), s) for i in a)
s

[1, 2, 3, 4, 5, 5]

In [26]:
test = pd.read_csv(os.getcwd() + "/INPUTS/test.csv", delimiter = ';')
test

,ID,DEPARTURE,ORIGIN,DESTINATION,SERVICE
0,1111241,07:00:00,634035.7291712608 5798370.933920433,633013.7429947932 5797843.393080657,PersonalVehicle
1,1168071,07:00:00,634495.9449736148 5809157.691576292,625022.1658398311 5803258.391749226,PersonalVehicle
2,1040303,07:00:02,626592.2832221197 5802782.201807101,627119.3937449739 5802857.838660972,PersonalVehicle
3,1135295,07:00:03,626373.3503814898 5795318.412810527,631520.9068714876 5795152.487842931,PersonalVehicle
4,1034483,07:00:04,627345.0816454801 5804263.484180098,622105.1529225953 5809623.637144065,PersonalVehicle
5,1118110,07:00:04,634121.8176268837 5795556.399203174,622519.4881247132 5795281.4729013005,PersonalVehicle
6,985601,07:00:05,632799.2457941913 5796925.2577712145,628408.4006607257 5802607.606304967,PersonalVehicle
7,396330,07:00:09,632741.7353963625 5806712.261097034,629039.4545156191 5804289.138790642,PersonalVehicle
8,1059793,07:00:10,629189.7559127033 5804304.686925133,630110.1454951254 5804014.466269681,PersonalVehicle
9,1124807,07:00:11,631939.4435178686 5799922.535666988,629076.2558468091 5803507.161900587,PersonalVehicle


In [27]:
test.to_csv('test_uber1.csv', sep = ';', index = False)

In [31]:
test = pd.read_csv(os.getcwd() + "/INPUTS/test.csv", delimiter = ';')
test_uber = percentage_uber(0.1, test)
test_uber

,ID,DEPARTURE,ORIGIN,DESTINATION,SERVICE
0,1111241,07:00:00,634035.7291712608 5798370.933920433,633013.7429947932 5797843.393080657,PersonalVehicle
1,1168071,07:00:00,634495.9449736148 5809157.691576292,625022.1658398311 5803258.391749226,UBER
2,1040303,07:00:02,626592.2832221197 5802782.201807101,627119.3937449739 5802857.838660972,PersonalVehicle
3,1135295,07:00:03,626373.3503814898 5795318.412810527,631520.9068714876 5795152.487842931,PersonalVehicle
4,1034483,07:00:04,627345.0816454801 5804263.484180098,622105.1529225953 5809623.637144065,PersonalVehicle
5,1118110,07:00:04,634121.8176268837 5795556.399203174,622519.4881247132 5795281.4729013005,PersonalVehicle
6,985601,07:00:05,632799.2457941913 5796925.2577712145,628408.4006607257 5802607.606304967,PersonalVehicle
7,396330,07:00:09,632741.7353963625 5806712.261097034,629039.4545156191 5804289.138790642,PersonalVehicle
8,1059793,07:00:10,629189.7559127033 5804304.686925133,630110.1454951254 5804014.466269681,PersonalVehicle
9,1124807,07:00:11,631939.4435178686 5799922.535666988,629076.2558468091 5803507.161900587,PersonalVehicle


In [73]:
test_uber_7h_9h = percentage_uber(0.03, test_all_in)
l = test_uber_7h_9h.SERVICE.tolist()
i=0
for t in l:
    if (t=='UBER'):
        i+=1
print(i)

363


In [74]:
len(test_uber_7h_9h)*0.05

605.0500000000001

In [84]:
test_all_in

,ID,DEPARTURE,ORIGIN,DESTINATION,SERVICE
0,1111241,07:00:00,634035.7291712608 5798370.933920433,633013.7429947932 5797843.393080657,PersonalVehicle
1,1168071,07:00:00,634418.6762973274 5805507.806356502,625022.1658398311 5803258.391749226,PersonalVehicle
2,1040303,07:00:02,626592.2832221197 5802782.201807101,627119.3937449739 5802857.838660972,PersonalVehicle
3,1135295,07:00:03,626373.3503814898 5795318.412810527,625217.5681790734 5799583.5073689055,PersonalVehicle
4,1034483,07:00:04,627345.0816454801 5804263.484180098,622105.1529225953 5795939.246536294,PersonalVehicle
...,...,...,...,...,...
12096,1021268,08:59:57,624509.1013833698 5803029.574844549,623917.2268032343 5804404.311049071,PersonalVehicle
12097,1089329,08:59:58,627184.6443263115 5799024.982075843,626527.037702565 5799611.930337115,PersonalVehicle
12098,1065005,08:59:59,627736.8477670127 5797076.775593162,627736.8477670127 5797076.775593162,PersonalVehicle
12099,1084683,08:59:59,627694.0701901402 5801284.886931946,627651.4043159435 5801055.3357389495,PersonalVehicle


In [85]:
test_all_in.to_csv('test_all_in_7h_9h.csv', sep = ';', index = False)

### Output of simulation

In [94]:
Output = pd.read_csv(os.getcwd() + "/OUTPUTS_all_in_city_7h_9h/path.csv", delimiter = ';')
Input = pd.read_csv(os.getcwd() + "/INPUTS/all_in_city_7h_9h.csv", delimiter = ';')

### Percentage of users found between 7h-9h

In [95]:
idOut = Output.ID.tolist()
idIn = Input.ID.to_list()
print ("percentage of users found: " + "{0:.0%}".format(len(set(idOut)) / len(set(idIn)))) 

percentage of users found: 100%


In [96]:
Output

,ID,COST,PATH,LENGTH,SERVICE
0,1111241,1092.478687,ORIGIN_108 m46193581 m1650696925 m3741600623 m...,6062.960154,PersonalVehicle
1,1168071,973.224424,ORIGIN_133 m46563125 m46560183 m46535871 m4653...,8014.674388,PersonalVehicle
2,1040303,149.882769,ORIGIN_120 m2020568942 DESTINATION_120,212.833532,PersonalVehicle
3,1135295,1061.776337,ORIGIN_92 m46020860 m46037691 m292823256 m5903...,7094.470527,PersonalVehicle
4,1034483,932.299808,ORIGIN_120 m46386982 m297095279 m46393384 m415...,8734.967502,PersonalVehicle
...,...,...,...,...,...
12077,1120808,833.400807,ORIGIN_108 m4034695571 DESTINATION_108,1183.429146,PersonalVehicle
12078,1021268,380.169939,ORIGIN_119 m296265502 DESTINATION_119,539.841313,PersonalVehicle
12079,1089329,23.837952,ORIGIN_106 m46186201 DESTINATION_106,33.849892,PersonalVehicle
12080,1065005,23.837952,ORIGIN_106 m46186201 DESTINATION_106,33.849892,PersonalVehicle


### 6h-20h

In [29]:
# Input 6h-20h
all_agent = pd.read_csv(os.getcwd() + "/INPUTS/test_6h_20h.csv", delimiter = ';')
agent = all_in_city(all_agent)
agent

,ID,DEPARTURE,ORIGIN,DESTINATION
0,1101033,06:00:00,632282.6418020834 5801707.366598499,632062.0681906 5802004.403086661
1,1089814,06:00:00,626721.0636507642 5798503.632446792,622309.477087811 5795980.005438486
2,993356,06:00:00,633038.5126445848 5809239.38811344,623762.9855951864 5800949.619838926
3,1218801,06:00:02,625613.743082539 5801708.914454374,631238.0300638282 5806467.613979217
4,378637,06:00:03,624437.178563774 5808224.330627002,633558.0593030814 5795612.615837722
...,...,...,...,...
394081,1092728,20:00:00,629686.0420486864 5802452.289847546,630601.1897881002 5802171.0732132085
394082,1118191,20:00:00,634737.5917461815 5795176.147458445,634974.4686226456 5795191.335723182
394083,1121694,20:00:00,635160.1586789726 5801507.30883727,632455.3112034239 5798521.229345774
394084,1125339,20:00:00,632120.0220604875 5799685.668698785,631765.334638351 5799745.078005204


In [30]:
(a,b,c,d,e) = in_out_city(all_agent)
print("in_in_city :", "{0:.0f}%".format(a), "in_out_city :", "{0:.0f}%".format(b), "out_in_city :", "{0:.0f}%".format(c), "out_out_city :","{0:.0f}%".format(d), "sum_out :", e)

in_in_city : 48% in_out_city : 13% out_in_city : 13% out_out_city : 26% sum_out : 277611


In [31]:
(a,b,c,d,e) = in_out_city(agent)
print("in_in_city :", "{0:.0f}%".format(a), "in_out_city :", "{0:.0f}%".format(b), "out_in_city :", "{0:.0f}%".format(c), "out_out_city :","{0:.0f}%".format(d), "sum_out :", e)

in_in_city : 100% in_out_city : 0% out_in_city : 0% out_out_city : 0% sum_out : 0


### Output of simulation

In [32]:
Output = pd.read_csv(os.getcwd() + "/OUTPUTS_all_in_city_6h_20h/path.csv", delimiter = ';')
Input = pd.read_csv(os.getcwd() + "/INPUTS/all_in_city_6h_20h.csv", delimiter = ';')

### Percentage of users found between 6h-20h

In [33]:
idOut = Output.ID.tolist()
idIn = Input.ID.to_list()
print ("percentage of users found: " + "{0:.0%}".format(len(set(idOut)) / len(set(idIn)))) 

percentage of users found: 100%


In [34]:
Output  # path

,ID,COST,PATH,LENGTH,SERVICE
0,1101033,196.851876,ORIGIN_121 m455971586 DESTINATION_121,279.529663,PersonalVehicle
1,1089814,979.741941,ORIGIN_106 m46186201 m46184457 m46184856 m4618...,10019.510223,PersonalVehicle
2,993356,2215.136926,ORIGIN_135 m46543900 m46535784 m46534856 m4653...,22025.871183,PersonalVehicle
3,1218801,1848.519529,ORIGIN_119 m296265502 m486079030 m46404228 m46...,19691.155453,PersonalVehicle
4,378637,2015.643273,ORIGIN_134 m46553321 m46554659 m46553671 m4655...,25516.825247,PersonalVehicle
...,...,...,...,...,...
393621,1092728,714.528530,ORIGIN_120 m46386982 m297095279 m46393384 m415...,5634.051242,PersonalVehicle
393622,1118191,1011.774480,ORIGIN_94 m45999429 DESTINATION_94,1436.719762,PersonalVehicle
393623,1121694,1849.008095,ORIGIN_122 m75538939 m46336665 m3175440817 m75...,11390.788250,PersonalVehicle
393624,1125339,401.796294,ORIGIN_107 m46191672 DESTINATION_107,570.550737,PersonalVehicle


In [48]:
# Input test 10 lines
test_agent = pd.read_csv(os.getcwd() + "/INPUTS/test_8h_10L.csv", delimiter = ';')
test = all_in_city(test_agent)
test

,ID,DEPARTURE,ORIGIN,DESTINATION
0,1103972,08:00:00,631535.1422327636 5803436.995133258,623384.9170858881 5806094.560615124
1,1115149,08:00:00,634129.4929278796 5798053.934760166,630781.8925280606 5800933.162112237
2,1035042,08:00:00,627636.2869598038 5804935.124059598,634339.1089068025 5798090.497493093
3,1107884,08:00:00,633830.4181053555 5797059.157535477,623045.5410589064 5801054.631141664
4,1104882,08:00:00,631656.3622370602 5803194.158888644,631889.0524407397 5802822.9000578
5,1082159,08:00:00,626840.4845576547 5801443.6167761525,626993.0846549141 5806473.503018918
6,1068917,08:00:00,628467.4701878248 5803027.810372679,628839.0794578048 5802805.310619754
7,1004465,08:00:00,633218.1255840912 5806592.060767567,630009.9025622855 5795731.260661593
8,1118776,08:00:00,633730.5369951553 5799394.341086689,634362.3799603812 5795839.954377281


In [88]:
x = 628050.3987994181
                        
y = 5805512.021205687

In [89]:
utm.to_latlon(x, y, 31, 'U')

(52.384873203248524, 4.881513100166867)

## Convert df to csv file

In [70]:
#test_all_in.to_csv('all_in_city_7h_9h.csv', sep = ';', index = False)

# Indicateurs pour les véhicules

In [90]:
os.getcwd()

'/Users/xinyun/Documents/Test_Amsterdam/Amsterdam'

In [117]:
input_veh = os.getcwd() + "/OUTPUTS_all_in_city_7h_9h/veh.csv"
veh = pd.read_csv(input_veh, delimiter = ";")
veh

,TIME,ID,TYPE,LINK,POSITION,SPEED,STATE,DISTANCE,PASSENGERS
0,07:00:00.00,12,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,1392154.0
1,07:00:00.00,11,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,406213.0
2,07:00:00.00,10,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,1132382.0
3,07:00:00.00,9,Car,NaN,NaN,13.800,SERVING,0.000,1131971.0
4,07:00:00.00,8,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,906014.0
...,...,...,...,...,...,...,...,...,...
135124,09:39:00.00,11937,Car,m3649782943 m3649776206,636107.624 5808759.607,11.499,STOP,23129.987,NaN
135125,09:39:00.00,11953,Car,m46505705 m3649023513,635555.966 5808140.312,11.499,SERVING,21976.780,1131162.0
135126,09:40:00.00,11805,Car,m46554659 m46553321,627671.981 5808943.170,11.499,STOP,26382.140,NaN
135127,09:40:00.00,11953,Car,m3649023513 m3649782943,636095.017 5808571.468,11.499,SERVING,22666.740,1131162.0


In [38]:
input_veh = os.getcwd() + "/OUTPUTS_all_in_city_6h_20h/veh.csv"
veh_6_20 = pd.read_csv(input_veh, delimiter = ";")
veh_6_20

,TIME,ID,TYPE,LINK,POSITION,SPEED,STATE,DISTANCE,PASSENGERS
0,06:00:00.00,16,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,390066.0
1,06:00:00.00,15,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,1088410.0
2,06:00:00.00,14,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,1089725.0
3,06:00:00.00,13,Car,NaN,NaN,13.800,SERVING,0.000,1126023.0
4,06:00:00.00,12,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,1088455.0
...,...,...,...,...,...,...,...,...,...
4506630,20:41:00.00,387443,Car,m46097566 m45999429,636713.741 5794072.569,11.498,STOP,22590.645,NaN
4506631,20:41:00.00,387447,Car,m46097566 m45999429,636713.741 5794072.569,11.498,STOP,22590.645,NaN
4506632,20:41:00.00,387476,Car,m46535784 m46543900,632169.433 5808699.725,11.498,STOP,22165.466,NaN
4506633,20:42:00.00,387423,Car,m46560183 m46563125,623460.671 5809244.355,11.500,SERVING,24467.840,1118497.0


## Trajectory  of vehicle

In [118]:
## Mouvement for ID = 174
def df_a_vehicle(df, idNumber):
    ID = df['ID'].tolist()
    indices = [i for i, x in enumerate(ID) if x == idNumber]
    dataframe = df.loc[indices]
    return dataframe

In [120]:
individual = df_a_vehicle(veh, 10)  # 7h-9h id : 11937, 10
individual

,TIME,ID,TYPE,LINK,POSITION,SPEED,STATE,DISTANCE,PASSENGERS
2,07:00:00.00,10,Car,m46186201 m46184457,NaN,13.800,SERVING,0.000,1132382.0
19,07:01:00.00,10,Car,m46190676 m4628394748,628141.572 5799173.996,11.497,SERVING,689.820,1132382.0
81,07:02:00.00,10,Car,m46194128 m253758837,628804.247 5799287.243,11.483,SERVING,1378.780,1132382.0
183,07:03:00.00,10,Car,m4977952665 m2601812857,628788.654 5799974.736,11.470,SERVING,2066.980,1132382.0
341,07:04:00.00,10,Car,m3277102402 m4566945243,628948.755 5800611.803,11.455,SERVING,2754.260,1132382.0
550,07:05:00.00,10,Car,m46248718 m46247804,629633.562 5800592.579,11.436,SERVING,3440.440,1132382.0
816,07:06:00.00,10,Car,m46239095 m1527945464,630171.809 5800307.045,11.417,SERVING,4125.440,1132382.0
1146,07:07:00.00,10,Car,m46231043 m46225963,630313.854 5800162.231,11.398,SERVING,4809.320,1132382.0
1531,07:08:00.00,10,Car,m46203628 m4628394740,630491.051 5799513.550,11.379,SERVING,5492.080,1132382.0
1978,07:09:00.00,10,Car,m1527945486 m4628394739,630839.202 5798940.641,11.357,SERVING,6173.500,1132382.0


In [41]:
trajet = df_a_vehicle(veh_6_20, 111)
trajet

,TIME,ID,TYPE,LINK,POSITION,SPEED,STATE,DISTANCE,PASSENGERS
29,06:01:00.00,111,Car,m46553321 m46554659,NaN,13.800,SERVING,0.000,1896790.0
199,06:02:00.00,111,Car,m46546563 m4620948066,628211.728 5808658.800,11.475,SERVING,688.480,1896790.0
370,06:03:00.00,111,Car,m4620948068 m46533230,628750.162 5808244.023,11.451,SERVING,1375.560,1896790.0
648,06:04:00.00,111,Car,m46521938 m46515974,628922.085 5807654.183,11.421,SERVING,2060.840,1896790.0
1037,06:05:00.00,111,Car,m46507117 m4620947756,629472.294 5807249.273,11.383,SERVING,2743.820,1896790.0
1572,06:06:00.00,111,Car,m46504858 m46505886,630004.405 5807283.992,11.339,SERVING,3424.140,1896790.0
2216,06:07:00.00,111,Car,m4620948074 m4620948073,630346.558 5807660.315,11.299,SERVING,4102.060,1896790.0
2996,06:08:00.00,111,Car,m4620947760 m46522619,630785.578 5807906.708,11.252,SERVING,4777.160,1896790.0
3928,06:09:00.00,111,Car,m3754752551 m943758453,631396.870 5807634.329,11.204,SERVING,5449.420,1896790.0
5008,06:10:00.00,111,Car,m3315109972 m3315108151,631945.604 5807251.378,11.155,SERVING,6118.700,1896790.0


In [42]:
l = individual.POSITION.tolist()
#coord = np.array([x.split() for x in l])
#coord
#l

## Distance

In [43]:
dist = veh.groupby(by = 'ID')[['DISTANCE']].agg(['sum','min','max'])
dist

DISTANCE                
              sum  min        max
ID                               
0       60770.330  0.0   8543.970
1      343589.961  0.0  20284.041
2           0.000  0.0      0.000
3           0.000  0.0      0.000
4      212614.328  0.0  15580.508
...           ...  ...        ...
34276  313725.349  0.0  19965.209
34277   13696.486  0.0   3815.586
34278   13730.595  0.0   3849.695
34279  254594.902  0.0  17974.262
34280   79762.256  0.0   9690.216

[34281 rows x 3 columns]

## Speed

In [44]:
speed = veh.groupby(by = 'ID')[['SPEED']].agg(['mean','min','max'])
speed

SPEED              
            mean     min   max
ID                            
0      11.203857  10.678  13.8
1      10.795303  10.442  13.8
2      13.800000  13.800  13.8
3      13.800000  13.800  13.8
4      10.888154  10.451  13.8
...          ...     ...   ...
34276  11.427065  10.887  13.8
34277  11.441857  10.887  13.8
34278  11.441857  10.887  13.8
34279  11.419500  10.887  13.8
34280  11.381125  10.887  13.8

[34281 rows x 3 columns]

## Travel time

In [45]:
time = veh['TIME'].tolist()
#[lambda x :( float(x[:2])*3600 + float(x[3:5])*60 + float(x[6:])) for x in time ]
turn_to_second = lambda x : float(x[:2])*3600 + float(x[3:5])*60 + float(x[6:]) - 9*3600  # Turn to second
for_all = lambda l, f : [f(i) for i in l]
travel_time = for_all(time, turn_to_second)
travel_time

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [46]:
veh['TIME'] = travel_time
veh

,TIME,ID,TYPE,LINK,POSITION,SPEED,STATE,DISTANCE,PASSENGERS
0,0.0,214,Car,m4621843969 m46442080,NaN,13.800,SERVING,0.000,954610.0
1,0.0,213,Car,NaN,NaN,13.800,SERVING,0.000,1031082.0
2,0.0,212,Car,m84486707 m300170947,NaN,13.800,SERVING,0.000,1067560.0
3,0.0,211,Car,m46443108 m46443464,NaN,13.800,SERVING,0.000,1044279.0
4,0.0,210,Car,m46436071 m46433671,NaN,13.800,SERVING,0.000,1035539.0
...,...,...,...,...,...,...,...,...,...
311494,9360.0,34179,Car,m46576068 m46586255,634053.373 5810478.954,11.499,STOP,22656.531,NaN
311495,9360.0,34266,Car,m46064509 m46059514,628792.243 5795542.933,11.499,SERVING,22455.080,1014484.0
311496,9420.0,34070,Car,m46584537 m46585577,629851.828 5810342.251,11.499,STOP,24829.012,NaN
311497,9420.0,34266,Car,m46057134 m46046972,629426.168 5795272.296,11.499,SERVING,23145.040,1014484.0


In [47]:
veh.groupby(by = 'ID').sum()

,TIME,SPEED,DISTANCE,PASSENGERS
ID,,,,
0,5460.0,156.854,60770.330,13251732.0
1,31680.0,356.245,343589.961,30673216.0
2,0.0,13.800,0.000,1033909.0
3,0.0,13.800,0.000,1047429.0
4,19500.0,283.092,212614.328,25121700.0
...,...,...,...,...
34276,258540.0,354.239,313725.349,30145800.0
34277,53340.0,80.093,13696.486,6629556.0
34278,53340.0,80.093,13730.595,6068742.0


## 

## bla